In [1]:
import numpy as np
import pandas as pd
import math

from qiskit import ClassicalRegister, QuantumRegister
from qiskit import QuantumCircuit, execute

In [2]:
qr = QuantumRegister(2)
cr = ClassicalRegister(2)
qc = QuantumCircuit(qr, cr)
qc.x(qr[0])

qc.measure(qr, cr)

job_sim = execute(qc, "local_qasm_simulator")
sim_result = job_sim.result()

print(sim_result.get_counts(qc))

{'01': 1024}


In [3]:
def ccz(circuit, q_reg):
    circuit.h(q_reg[2])
    circuit.ccx(q_reg[0], q_reg[1], q_reg[2])
    circuit.h(q_reg[2])

In [4]:
def oracle(circuit, q_reg, n):
    if len(q_reg) != 3:
        raise ValueError('This is a 3 qubit oracle only!')
    if (n < 0) or (n > 7):
        raise ValueError('3 qubits means 0 <= n <= 7')
    
    qbits_to_flip = [q_bit for flip_q, q_bit in zip('{0:b}'.format(n+8)[1:], q_reg)
                     if int(flip_q) == 0]
    
    for qbit in qbits_to_flip:
        circuit.x(qbit)
    
    ccz(circuit, q_reg)
    
    for qbit in qbits_to_flip:
        circuit.x(qbit)

In [5]:
def all_hs(circuit, q_reg):
    for q_bit in q_reg:
        circuit.h(q_bit)

In [6]:
def diffusion_operator(circuit, q_reg):
    all_hs(circuit, q_reg)
    for q_bit in q_reg:
        circuit.x(q_bit)
    ccz(circuit, q_reg)
    for q_bit in q_reg:
        circuit.x(q_bit)
    all_hs(circuit, q_reg)

In [28]:
qr = QuantumRegister(3)
cr = ClassicalRegister(3)
qc = QuantumCircuit(qr, cr)

n = 3

all_hs(qc, qr)

for m in range(0, 2):
    oracle(qc, qr, n)
    diffusion_operator(qc, qr)

qc.measure(qr, cr)

job_sim = execute(qc, "local_qasm_simulator")
sim_result = job_sim.result()

print(sim_result.get_counts(qc))

{'000': 7, '001': 8, '010': 6, '011': 6, '100': 12, '101': 5, '110': 969, '111': 11}
